In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/QMIC_App

/content/drive/MyDrive/QMIC_App


In [3]:
!pip install transformers
!pip install tqdm
!pip install langdetect
!pip install arabert
!pip install openpyxl


In [5]:
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, TensorDataset
from langdetect import detect
from concurrent.futures import ProcessPoolExecutor
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from arabert.preprocess import ArabertPreprocessor

In [6]:
# df = pd.read_csv('CleanedQuData.csv', sep='|', names=['content', 'path'])
df = pd.read_excel('website_dataAr.xlsx')

# Check for NaN values in 'content' column
print(df['content'].isna().sum())

# Drop rows with NaN values
df = df.dropna(subset=['content'])

#Detect language
df['lang'] = df['content'].apply(lambda x: detect(x))
df_english = df[df['lang'] == 'en']
df_arabic = df[df['lang'] == 'ar']

2


In [7]:
df_arabic = df_arabic.reset_index(drop=True)
df_english = df_english.reset_index(drop=True)
df_arabic

,context,content,path,lang
0,NaN,سيجمع المؤتمر الدولي الرابع للاستدامة (ISC2023...,NaN,ar
1,NaN,أيماناً من مركز خدمة المجتمع والتعليم المستمر ...,"الرئيسية,مركز خدمة المجتمع والتعليم المستمر,خد...",ar
2,NaN,تهدف برامج خدمة المجتمع في مركز خدمة المجتمع و...,"الرئيسية,مركز خدمة المجتمع والتعليم المستمر",ar
3,NaN,يقدم مركز خدمة المجتمع والتعليم المستمر (CCE) ...,"الرئيسية,مركز خدمة المجتمع والتعليم المستمر",ar
4,NaN,قام مركز خدمة المجتمع والتعليم المستمر بتنظيم ...,"الرئيسية,مركز خدمة المجتمع والتعليم المستمر,مو...",ar
...,...,...,...,...
2328,مركز ابن خلدون للدراسات الإنسانية والاجتماعية,n.alshamari@qu.edu.qa4403 4441سكن طالبات جامعة...,"الرئيسية,البحوث والدراسات العليا,عن المكتب",ar
2329,دار نشر جامعة قطر,qupress@qu.edu.qa4403 7198سكن طالبات جامعة قطر...,"الرئيسية,البحوث والدراسات العليا,عن المكتب",ar
2330,محطة البحوث الزراعية,ars@qu.edu.qa4403 4564تقع محطة البحوث الزراعية...,"الرئيسية,البحوث والدراسات العليا,عن المكتب",ar
2331,مكتب الاتصال والشراكات,VPRGS.ECO@qu.edu.qa4403 3259مجمع البحوث (H10)،...,"الرئيسية,البحوث والدراسات العليا,عن المكتب",ar


In [ ]:
df_english

,context,content,path,lang
0,NaN,يرجى العلم ان القبول تنافسي حسب المقاعد: يمكن ...,"كلية الإدارة والإقتصاد,المكاتب,مكاتب شؤون الطلبة",en
1,النماذج,"Program Mission, Objectives and Learning Outco...","كلية الآداب والعلوم,مكتب عميد كلية الآداب والع...",en
2,نظام التقييم عبر الإنترنت,Online Assessment System Assessors User Manual...,"كلية الآداب والعلوم,مكتب عميد كلية الآداب والع...",en
3,NaN,Dr. Sara earned her Bachelor of Science in Pha...,NaN,en
4,NaN,Qatar University Research Complex is a purpose...,"Student Services,Student Auxiliary Services,Ca...",en
5,NaN,"With the start of this current academic year, ...",NaN,en
6,NaN,Program period: Two years of full time study (...,"College of Arts and Sciences,Departments,Arabi...",en
7,NaN,يشكّل مجمع البحوث في جامعة قطر مرفقاً حديثاً ي...,"الرئيسية,عن الجامعة,غرفة الأخبار,جامعة قطر",en
8,NaN,يشكّل مجمع البحوث في جامعة قطر مرفقاً حديثاً ي...,"الرئيسية,عن الجامعة,غرفة الأخبار,معهد البحوث ا...",en
9,2023,ُجامعة قطر تُنظم فعالية بحثية بعنوان استدامة و...,"الرئيسية,البحوث والدراسات العليا,البحوث الاجتم...",en


In [8]:
df_arabic.shape

(2333, 4)

In [9]:
# This is just for tqdm to work with pandas' apply function
tqdm.pandas()

In [ ]:
# !git clone https://github.com/aub-mind/arabert.git
model_name2 = "bert-base-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name2, keep_emojis=False)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|██████████| 241M/241M [00:21<00:00, 11.4MiB/s]


[2023-10-08 10:37:27,969 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [ ]:
#English
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)
model.eval()
model.to('cuda')


#Or work in parallel on a GPU

def summarize_batch(batch):
    inputs = tokenizer.batch_encode_plus(batch, return_tensors="pt", max_length=1024, truncation=True, padding='max_length')
    inputs = {k: v.to('cuda') for k, v in inputs.items()}  # Send inputs to GPU

    summary_ids = model.generate(**inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    summaries = [tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids]
    return summaries

# Convert df['content'] to a list
texts = df_english['content'].tolist()

# Create a DataLoader
batch_size = 16  # Adjust based on the size of your data and GPU memory
dataloader = DataLoader(texts, batch_size=batch_size, shuffle=False)

# Process in batches
all_summaries = []
for batch in tqdm(dataloader):
    all_summaries.extend(summarize_batch(batch))

df_english['summary'] = all_summaries

#Save English summary
df[['content', 'path', 'summary']].to_csv('summarized_data.csv', sep='|', index=False)



In [ ]:
#Without parallasim
# def summarize_text(text):
#     # Convert the text to a string and check if it's not empty
#     text = str(text)
#     if not text:
#         return ""

#     inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
#     summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary

# # Apply the summarization function to your content column with a progress bar
# df['summary'] = df['content'].progress_apply(summarize_text)

# # Apply the summarization function to your content column
# # df['summary'] = df['content'].apply(summarize_text)

In [10]:
df_arabic = df_arabic[df_arabic['content'].notna() & (df_arabic['content'] != '')]
df_arabic['content'] = df_arabic['content'].astype(str)
df_arabic = df_arabic[df_arabic['content'].notna() & (df_arabic['content'] != '')]
df_arabic = df_arabic[df_arabic['content'].str.strip() != '']

In [12]:
df_arabic.shape

(2333, 4)

In [13]:
df_arabic

,context,content,path,lang
0,NaN,سيجمع المؤتمر الدولي الرابع للاستدامة (ISC2023...,NaN,ar
1,NaN,أيماناً من مركز خدمة المجتمع والتعليم المستمر ...,"الرئيسية,مركز خدمة المجتمع والتعليم المستمر,خد...",ar
2,NaN,تهدف برامج خدمة المجتمع في مركز خدمة المجتمع و...,"الرئيسية,مركز خدمة المجتمع والتعليم المستمر",ar
3,NaN,يقدم مركز خدمة المجتمع والتعليم المستمر (CCE) ...,"الرئيسية,مركز خدمة المجتمع والتعليم المستمر",ar
4,NaN,قام مركز خدمة المجتمع والتعليم المستمر بتنظيم ...,"الرئيسية,مركز خدمة المجتمع والتعليم المستمر,مو...",ar
...,...,...,...,...
2328,مركز ابن خلدون للدراسات الإنسانية والاجتماعية,n.alshamari@qu.edu.qa4403 4441سكن طالبات جامعة...,"الرئيسية,البحوث والدراسات العليا,عن المكتب",ar
2329,دار نشر جامعة قطر,qupress@qu.edu.qa4403 7198سكن طالبات جامعة قطر...,"الرئيسية,البحوث والدراسات العليا,عن المكتب",ar
2330,محطة البحوث الزراعية,ars@qu.edu.qa4403 4564تقع محطة البحوث الزراعية...,"الرئيسية,البحوث والدراسات العليا,عن المكتب",ar
2331,مكتب الاتصال والشراكات,VPRGS.ECO@qu.edu.qa4403 3259مجمع البحوث (H10)،...,"الرئيسية,البحوث والدراسات العليا,عن المكتب",ar


In [29]:
#Arabic dataset

from transformers import GPT2TokenizerFast, pipeline
from transformers import GPT2LMHeadModel
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel
from arabert.preprocess import ArabertPreprocessor
from tqdm import tqdm
import torch
from transformers import MBartForConditionalGeneration, MBartTokenizer
# Assuming you have already loaded df_arabic
# df_arabic = pd.read_csv("your_data.csv") or any other method

MODEL_NAME='facebook/mbart-large-50'
arabert_prep = ArabertPreprocessor(model_name=MODEL_NAME)

# model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
# tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_NAME)

tokenizer = MBartTokenizer.from_pretrained(MODEL_NAME)
model = MBartForConditionalGeneration.from_pretrained(MODEL_NAME)


# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_summary_batch(texts):
    # Tokenize the input text as a batch
    encoding = tokenizer.batch_encode_plus(texts, return_tensors='pt', truncation=True, max_length=150, padding='max_length')
    tokenized_texts = {key: val.to(device) for key, val in encoding.items()}

    cleaned_texts = [arabert_prep.preprocess(text) for text in texts]

    # Generate the summary
    summary_ids = model.generate(tokenized_texts["input_ids"], num_beams=4, max_length=100, early_stopping=True, forced_bos_token_id=tokenizer.lang_code_to_id["ar_AR"])

    # Decode the generated summaries for each item in the batch
    # summaries = [tokenizer.decode(summary_id, skip_special_tokens=True) for summary_id in summary_ids]
    summaries = [tokenizer.decode(summary_id, skip_special_tokens=True).replace('<pad>', '').strip() for summary_id in summary_ids]

    return summaries





# Process the dataframe in batches
BATCH_SIZE = 8
summary_list = []

for i in tqdm(range(0, len(df_arabic), BATCH_SIZE), desc="Generating Summaries"):
    batch_texts = df_arabic['content'].iloc[i:i+BATCH_SIZE].tolist()

    batch_summaries = generate_summary_batch(batch_texts)
    summary_list.extend(batch_summaries)

df_arabic['summary'] = summary_list


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.
Generating Summaries: 100%|██████████| 292/292 [09:42<00:00,  1.99s/it]


In [35]:
# Use the data from 'context' column, and fill missing values with data from 'summary' column
df_arabic['merged'] = df_arabic['context'].fillna(df_arabic['summary'])

# If you want to replace the 'context' column with this merged column, you can do:
df_arabic['context'] = df_arabic['merged']
# And if you want to remove the 'merged' column afterwards:
df_arabic.drop('merged', axis=1, inplace=True)
df_arabic.drop('lang', axis=1, inplace=True)

In [36]:
df_arabic[['content', 'path', 'summary', 'context']].to_csv('summarized_dataAra.csv', sep='|', index=False)

df_arabic.to_excel("summarized_dataAra.xlsx", index=False, engine='openpyxl')
# df_arabic[['content', 'path', 'summary', 'context']].to_excel('summarized_dataAra2.xlsx', index=False, engine='openpyxl')


In [43]:
df_arabic

,context,content,path,summary
0,استضافة جامعة قطر المؤتمر الدولي الرابع للاستد...,سيجمع المؤتمر الدولي الرابع للاستدامة (ISC2023...,NaN,استضافة جامعة قطر المؤتمر الدولي الرابع للاستد...
1,إيماناً من مركز خدمة المجتمع والتعليم المستمر ...,أيماناً من مركز خدمة المجتمع والتعليم المستمر ...,"الرئيسية,مركز خدمة المجتمع والتعليم المستمر,خد...",إيماناً من مركز خدمة المجتمع والتعليم المستمر ...
2,هدف برامج خدمة المجتمع في مركز خدمة المجتمع وا...,تهدف برامج خدمة المجتمع في مركز خدمة المجتمع و...,"الرئيسية,مركز خدمة المجتمع والتعليم المستمر",هدف برامج خدمة المجتمع في مركز خدمة المجتمع وا...
3,مركز خدمة المجتمع والتعليم المستمر (CCE) دورات...,يقدم مركز خدمة المجتمع والتعليم المستمر (CCE) ...,"الرئيسية,مركز خدمة المجتمع والتعليم المستمر",مركز خدمة المجتمع والتعليم المستمر (CCE) دورات...
4,مركز خدمة المجتمع والتعليم المستمر بتنظيم وتقد...,قام مركز خدمة المجتمع والتعليم المستمر بتنظيم ...,"الرئيسية,مركز خدمة المجتمع والتعليم المستمر,مو...",مركز خدمة المجتمع والتعليم المستمر بتنظيم وتقد...
...,...,...,...,...
2328,مركز ابن خلدون للدراسات الإنسانية والاجتماعية,n.alshamari@qu.edu.qa4403 4441سكن طالبات جامعة...,"الرئيسية,البحوث والدراسات العليا,عن المكتب",n.alshamari@qu.edu.qa4403 4441سكن طالبات جامعة...
2329,دار نشر جامعة قطر,qupress@qu.edu.qa4403 7198سكن طالبات جامعة قطر...,"الرئيسية,البحوث والدراسات العليا,عن المكتب",qupress@qu.edu.qa4403 7198سكن طالبات جامعة قطر...
2330,محطة البحوث الزراعية,ars@qu.edu.qa4403 4564تقع محطة البحوث الزراعية...,"الرئيسية,البحوث والدراسات العليا,عن المكتب",ars@qu.edu.qa4403 4564تقع محطة البحوث الزراعية...
2331,مكتب الاتصال والشراكات,VPRGS.ECO@qu.edu.qa4403 3259مجمع البحوث (H10)،...,"الرئيسية,البحوث والدراسات العليا,عن المكتب",VPRGS.ECO@qu.edu.qa4403 3259مجمع البحوث (H10) ...


In [ ]:
# def generate_summary_batch(texts):




#     # # Tokenize, truncate, and preprocess texts in a batch
#     # tokenized_texts = tokenizer(texts, return_tensors="pt", truncation=True, max_length=150, padding='max_length').to(device)
#     # cleaned_texts = [arabert_prep.preprocess(tokenizer.decode(txt, skip_special_tokens=True)) for txt in tokenized_texts["input_ids"]]

#     # # Move tokenized texts to the same device as the model
#     # tokenized_texts = {key: val.to(device) for key, val in tokenized_texts.items()}

#     # # Generate summaries directly using the model
#     # summaries_ids = model.generate(
#     #     tokenized_texts["input_ids"],
#     #     attention_mask=tokenized_texts["attention_mask"],
#     #     # pad_token_id=tokenizer.eos_token_id,
#     #     num_beams=4,
#     #     max_length=300,
#     #     top_p=0.9,
#     #     early_stopping=True,
#     #     repetition_penalty=3.0,
#     #     no_repeat_ngram_size=3,
#     #     forced_bos_token_id=tokenizer.lang_code_to_id["ar_AR"]
#     # )

#     # # summaries = [tokenizer.decode(gen_id, skip_special_tokens=True) for gen_id in summaries_ids]
#     # summaries = tokenizer.decode(summaries_ids.squeeze(), skip_special_tokens=True)

#     return summaries

In [ ]:
#google/mt5-base
# moussaKam/AraBART
#malmarjeh/mbert2mbert-arabic-text-summarization

In [45]:
!git config --global user.email "omama_hamad2003@hotmail.com"
!git config --global user.name "Omama"


In [46]:
!git clone https://github.com/omama1982/Arabic-Summary.git


Cloning into 'Arabic-Summary'...


In [48]:
%cd Arabic-Summary


/content/drive/MyDrive/QMIC_App/Arabic-Summary


In [56]:
!mv /content/drive/MyDrive/QMIC_App/Arabic-Summary-mbart.ipynb .


mv: cannot stat '/content/drive/MyDrive/QMIC_App/Arabic-Summary-mbart.ipynb': No such file or directory


In [ ]:
!git add .
!git commit -m "Your commit message here"
